![](https://fenc.wang/imgs/2020/08/3565654bfbe9ffa0.png)

In [10]:
import tushare as ts
import pandas as pd

#设置完整显示数据
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

pro = ts.pro_api('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')

def cagr(start_value, end_value, num_periods):
    return (end_value / start_value) ** (1 / (num_periods - 1)) - 1

In [11]:
#时间段
start_date = '20151231'

#股票池
stock_list = ['爱旭股份', '晶澳科技', '协鑫集成', '中环股份', '东方日升', '中来股份', '亿晶光电', '天合光能', '向日葵', '隆基股份', '通威股份', '正泰电器', '航天机电', '拓日新能']

In [12]:
result_list = []
stock_basic = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name')
stock_selections = stock_basic[stock_basic['name'].isin(stock_list)]

for ticker, name in zip(stock_selections['ts_code'], stock_selections['name']):
    result_dict = {}
    total_revenue = pro.income(ts_code=ticker,fields='ts_code,end_date,total_revenue')
    total_revenue.drop_duplicates(subset=['ts_code','end_date'],keep='first',inplace=True)
    total_revenue.set_index('end_date', inplace=True)
    total_revenue_yrs = total_revenue.filter(like='1231', axis=0)
    total_revenue_yrs = total_revenue_yrs[total_revenue_yrs.index >= start_date]
    
    #计算CAGR
    start_value = float(total_revenue_yrs.iloc[-1]['total_revenue'])
    end_value = float(total_revenue_yrs.iloc[0]['total_revenue'])
    num_periods = len(total_revenue_yrs['total_revenue'])
    cagr_y = cagr(start_value, end_value, num_periods)
    
    result_dict['ts_code'] = ticker
    result_dict['股票名称'] = name
    result_dict['年复合增长率'] = cagr_y * 100
    result_dict['周期'] = num_periods - 1
    result_list.append(result_dict)
    
df = pd.DataFrame(result_list)
df.sort_values(by='年复合增长率', ascending=False)

,ts_code,股票名称,年复合增长率,周期
10,600732.SH,爱旭股份,144.64,5
2,002459.SZ,晶澳科技,140.34,5
11,601012.SH,隆基股份,55.79,5
6,300393.SZ,中来股份,47.27,5
0,002129.SZ,中环股份,30.49,5
8,600438.SH,通威股份,25.71,5
5,300118.SZ,东方日升,25.02,5
12,601877.SH,正泰电器,22.56,5
1,002218.SZ,拓日新能,13.32,5
7,600151.SH,航天机电,8.57,5


In [13]:
pe = pro.daily_basic(trade_date='20210609', fields='ts_code,pe,pe_ttm, total_mv')
peg_data = df.merge(pe, on=['ts_code'])
peg_data['peg'] = peg_data['pe'].div(peg_data['年复合增长率']) 
peg_data['peg_ttm'] = peg_data['pe_ttm'].div(peg_data['年复合增长率']) 


In [14]:
peg_data.sort_values(by='peg', ascending=False)

,ts_code,股票名称,年复合增长率,周期,pe,pe_ttm,total_mv,peg,peg_ttm
7,600151.SH,航天机电,8.57,5,80.03,56.22,"1,372,579.44",9.34,6.56
13,688599.SH,天合光能,6.82,4,28.75,27.05,"3,534,257.07",4.22,3.97
5,300118.SZ,东方日升,25.02,5,78.94,289.97,"1,305,169.19",3.15,11.59
0,002129.SZ,中环股份,30.49,5,81.88,64.70,"8,916,804.03",2.69,2.12
1,002218.SZ,拓日新能,13.32,5,31.55,32.22,"522,972.71",2.37,2.42
8,600438.SH,通威股份,25.71,5,43.05,37.78,"15,530,341.23",1.67,1.47
6,300393.SZ,中来股份,47.27,5,71.81,54.28,"694,092.63",1.52,1.15
11,601012.SH,隆基股份,55.79,5,43.91,40.86,"37,550,426.22",0.79,0.73
12,601877.SH,正泰电器,22.56,5,10.31,10.67,"6,624,068.51",0.46,0.47
2,002459.SZ,晶澳科技,140.34,5,34.31,37.52,"5,168,753.53",0.24,0.27


![](https://fenc.wang/imgs/2020/08/3565654bfbe9ffa0.png)